# Graph Neural Network Training

Creation of graphs were done with 
```tools/training/GraphCreationModel.py```
Files can be found in: 
```/eos/cms/store/user/folguera/L1TMuon/INTREPID/Graphs_v240725_241015/```
in two flavours, with "all" connected layers and with up to "3-neighbour" layers connections. 

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import os, sys

import matplotlib.pyplot as plt


In [2]:
from models import GATRegressor

## Load data

In [3]:
## check if EOS folder exists otherwise use local folder
if os.path.exists("/eos/cms/store/user/folguera/L1TMuon/INTREPID/Graphs_v240725_241015/"):
    GraphDIR = "/eos/cms/store/user/folguera/L1TMuon/INTREPID/Graphs_v240725_241015/"
else:
    GraphDIR = "../graph_folder/"
using_only = 5  ## number of files used
print(GraphDIR)

../graph_folder/


In [4]:
Allgraphs = []
all_files = os.listdir(GraphDIR)

# Filter for .pkl files
pkl_files = [f for f in all_files if f.endswith('.pkl') and '_3_' in f]
print(f"Using files: {pkl_files}")
if not pkl_files:
    print("No .pkl files found in the directory.")
    sys.exit()

Using files: ['vix_graph_3_15Oct_onlypt_009.pkl', 'vix_graph_3_15Oct_onlypt_008.pkl', 'vix_graph_3_15Oct_onlypt_005.pkl', 'vix_graph_3_15Oct_onlypt_004.pkl', 'vix_graph_3_15Oct_onlypt_006.pkl', 'vix_graph_3_15Oct_onlypt_007.pkl', 'vix_graph_3_15Oct_onlypt_003.pkl', 'vix_graph_3_15Oct_onlypt_002.pkl', 'vix_graph_3_15Oct_onlypt_001.pkl']


In [5]:
count_files = 0
for pkl_file in pkl_files:
    if count_files >= using_only: break
    file_path = os.path.join(GraphDIR, pkl_file)
    print(f"Loading file: {pkl_file}")
    with open(file_path, 'rb') as file:
        graphfile = torch.load(file)
        Allgraphs.append(graphfile)
    count_files+=1

Loading file: vix_graph_3_15Oct_onlypt_009.pkl


/var/folders/08/4qyj2h792lv7kk8b3phm_j000000gn/T/ipykernel_10486/3026860541.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graphfile = torch.load(file)


Loading file: vix_graph_3_15Oct_onlypt_008.pkl
Loading file: vix_graph_3_15Oct_onlypt_005.pkl
Loading file: vix_graph_3_15Oct_onlypt_004.pkl
Loading file: vix_graph_3_15Oct_onlypt_006.pkl


In [6]:
BatchSize=64

Graphs_for_training = sum(Allgraphs, [])
Graphs_for_training_reduced = Graphs_for_training
Graphs_for_training_filtered = [g for g in Graphs_for_training_reduced if g.edge_index.size(1) > 0]  # remove empty graphs

# remove extra dimenson in y
print(f"Total Graphs: {len(Graphs_for_training)}")
print(f"Total Graphs after filtering: {len(Graphs_for_training_filtered)}")
for i in range(0, len(Graphs_for_training_filtered)):
    Graphs_for_training_filtered[i].y = Graphs_for_training_filtered[i].y.mean(dim=0)

# Train and test split:
events = len(Graphs_for_training_filtered)
ntrain = int((events * 0.7) / BatchSize) * BatchSize  # to have full batches
print(f"Training events: {ntrain}")
train_dataset = Graphs_for_training_filtered[:ntrain]
test_dataset = Graphs_for_training_filtered[ntrain:ntrain * 2]

print("====================================")
print("Example of data:")
print(train_dataset[0].x)
print(train_dataset[0].edge_index)
print(train_dataset[0].deltaPhi)
print(train_dataset[0].deltaEta)
print(train_dataset[0].y)
print("====================================")


# Load data
train_loader = DataLoader(train_dataset, batch_size=BatchSize, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)



Total Graphs: 279461
Total Graphs after filtering: 279332
Training events: 195520
Example of data:
tensor([[1.0005e+00, 1.7940e+03, 4.3113e+02, 0.0000e+00, 3.0000e+00],
        [9.4612e-01, 1.7750e+03, 4.1368e+02, 1.0000e+01, 5.0000e+00],
        [8.9175e-01, 1.7780e+03, 4.4868e+02, 1.1000e+01, 5.0000e+00],
        [9.7875e-01, 1.7890e+03, 1.0766e+03, 6.0000e+00, 9.0000e+00],
        [1.0005e+00, 1.7810e+03, 1.0914e+03, 1.5000e+01, 5.0000e+00]],
       dtype=torch.float64)
tensor([[0, 0, 1, 1, 2, 2, 3, 4],
        [1, 2, 0, 2, 0, 1, 4, 3]])
tensor([-19, -16, -19,   3, -16,   3,  -8,  -8])
tensor([-0.0544, -0.1088, -0.0544, -0.0544, -0.1088, -0.0544,  0.0217,  0.0217])
tensor([-64.5507])


In [ ]:
model = GATRegressor(num_node_features, num_edge_features, hidden_dim, output_dim).to(self.device)
optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LearningRate, weight_decay=0.75)
print("Model initialized")
print(model)

### Training loop

In [7]:
num_node_features = 5
hidden_dim = BatchSize
output_dim = 1
LearningRate=0.0005
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")
model = GATRegressor(num_node_features, hidden_dim, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LearningRate, weight_decay=0.75)
loss_fn = torch.nn.MSELoss()
print("Model initialized")
print(model)


Using device: cpu
Model initialized
GATRegressor(
  (conv1): GATConv(5, 64, heads=1)
  (conv2): GATConv(64, 64, heads=1)
  (fc1): Linear(in_features=128, out_features=1, bias=True)
)


In [8]:
train_losses = []
test_losses = []
#
# path = "/eos/cms/store/user/folguera/L1TMuon/INTREPID/Model_v240725_241015/"
path = "../model_folder/"
if not os.path.exists(path):
    os.makedirs(path)

In [43]:
def train():
    model.train()
    total_loss = 0
    i = 0
    for data in train_loader:
        data = data.to(device)  # Mueve los datos al dispositivo
        optimizer.zero_grad()
        data.y = data.y.float()  # Asegurarse de que los datos sean float32
        data.x = data.x.float()  # Asegurarse de que los datos sean float32
        out = model(data)
        try:
            loss = loss_fn(out, data.y.view(out.size()))
        except KeyError as e:
            print(f"KeyError: {e}")
            # Maneja el error o proporciona un valor por defecto
            loss = torch.tensor(0.0, device=device)

        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    return total_loss / len(train_loader.dataset)

def test():
    with torch.no_grad():
        model.eval()
        total_loss = 0
        for data in test_loader:
            data = data.to(device)
            data.x = data.x.float()  # Asegurarse de que los datos sean float32
            data.y = data.y.float()  # Asegurarse de que los datos sean float32
            out = model(data)
            loss = loss_fn(out, data.y.view(out.size()))
            total_loss += float(loss)
    return total_loss / len(test_loader.dataset)


print("Start training...")
for epoch in range(1000):
    train_loss = train()
    test_loss = test()
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    if epoch == 0:
        print(f'Epoch: {epoch + 1:02d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}')
        torch.save(test_loss, f"{path}/testloss_{epoch + 1}.pt")
        torch.save(train_loss, f"{path}/trainloss_{epoch + 1}.pt")
    elif (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1:02d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}')
        torch.save(model, f"{path}/model_{epoch + 1}.pth")
        torch.save(test_loss, f"{path}/testloss_{epoch + 1}.pt")
        torch.save(train_loss, f"{path}/trainloss_{epoch + 1}.pt")

        plt.plot(train_losses, "b", label="Train loss")
        plt.plot(test_losses, "k", label="Test loss")
        plt.yscale('log')
        plt.savefig(f"{path}/loss_plot.png")


Start training...
Epoch: 01, Train loss: 3.8789, Test loss: 3.8188


KeyboardInterrupt: 

## Plotting and checking
Now we need to plot everything and check the predicted momentum

In [41]:
class PlotRegression:
    def __init__(self, model, test_loader, batch_size):
        self.model = model
        self.test_loader = test_loader
        self.batch_size = batch_size
        self.pt_pred_arr = []
        self.pt_truth_arr = []

    def evaluate(self):
        with torch.no_grad():
            for data in self.test_loader:
                out = self.model(data)
                for item in range(0, out.size(0)):
                    vector_pred = out[item]
                    vector_real = data[item].y
                    self.pt_pred_arr.append(vector_pred.item())
                    self.pt_truth_arr.append(vector_real.item())

    def plot_regression(self, output_dir):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.clf()
        plt.hist(self.pt_truth_arr, bins=100, color='skyblue', alpha=0.5, label="truth")
        plt.hist(self.pt_pred_arr, bins=100, color='g', alpha=0.5, label="prediction")
        plt.legend()
        plt.savefig(os.path.join(output_dir, "pt_regression.png"))
        plt.clf()


        plt.plot(self.pt_truth_arr, self.pt_pred_arr, 'o')
        plt.xlabel("Truth")
        plt.ylabel("Prediction")
        plt.savefig(os.path.join(output_dir, "pt_regression_scatter.png"))
        plt.clf()

        # plot difference between truth and prediction
        diff = [x - y for x, y in zip(self.pt_truth_arr, self.pt_pred_arr)]
        plt.hist(diff, bins=100, color='r', alpha=0.5, label="difference")
        plt.legend()
        plt.savefig(os.path.join(output_dir, "pt_regression_diff.png"))
        plt.clf()
        



In [42]:
model = torch.load("../model_folder/model_100.pth")
            
evaluator = PlotRegression(model, test_loader, batch_size=BatchSize)
evaluator.evaluate()
evaluator.plot_regression(output_dir="../model_folder/")


/var/folders/08/4qyj2h792lv7kk8b3phm_j000000gn/T/ipykernel_10486/2111270606.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("../model_folder/model_100

<Figure size 640x480 with 0 Axes>